In [ ]:
import pandas as pd
from neuralprophet import NeuralProphet
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import seaborn as sns
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
df = pd.read_csv('austin_weather.csv')
df.tail()

In [ ]:
sns.pairplot(df[['SeaLevelPressureAvgInches', 'TempAvgF', 'SeaLevelPressureHighInches', 'TempHighF']], diag_kind='kde')

In [ ]:
df.Date.unique()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df ['Date'] = pd.to_datetime(df ['Date'])
df.tail()

In [ ]:
df.dtypes

In [ ]:
df = df.astype({'SeaLevelPressureAvgInches':'float'})
print(df.dtypes)

In [ ]:
plt.plot(df ['SeaLevelPressureAvgInches'])
plt.show()

In [ ]:
## Data and memory processing using Tensorflow

In [ ]:
df_feat = df.copy()
df_labels = df_feat.pop('SeaLevelPressureAvgInches')
df_feat.pop('Date')

In [ ]:
df_feat = np.asarray(df_feat).astype(np.float32)
df_feat

In [ ]:
new_model = tf.keras.Sequential([
  layers.Dense(100),
  layers.Dense(3)
])

new_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())
new_model.fit(df_feat, df_labels, epochs=10)

In [ ]:
### Normalizing the data with Tensorflow

In [ ]:
train_dataset = df.copy()
test_dataset = df.copy()

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()
train_features.pop('Date')
train_labels = train_features.pop('SeaLevelPressureAvgInches')
test_labels = test_features.pop('SeaLevelPressureAvgInches')

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

In [ ]:
sealevel = np.array(train_features['SeaLevelPressureHighInches'])

sealevel_normalizer = layers.Normalization(input_shape=[1,], axis=None)
sealevel_normalizer.adapt(sealevel)

sealevel_model = tf.keras.Sequential([
    sealevel_normalizer,
    layers.Dense(units=1)
])

sealevel_model.summary()

In [ ]:
## Perdicting using tensorflow

In [ ]:
sealevel_model.predict(sealevel[:200])
fea = sealevel_model.predict(sealevel[:200]).copy()

In [ ]:
plt.plot(fea)
plt.xlabel("Years")
plt.ylabel("Sea Level Inches")

In [ ]:
#### Below is for Training and perdicting the data using neural networks

In [ ]:
new_column = test_features[['Date', 'SeaLevelPressureLowInches']] 
new_column.dropna(inplace=True)
new_column.columns = ['ds', 'y'] 
new_column.tail()

In [ ]:
n = NeuralProphet()
model = n.fit(new_column, freq='D')

In [ ]:
future = n.make_future_dataframe(new_column, periods=200000)
forecast = n.predict(future)
forecast.tail()

In [ ]:
plot = n.plot(forecast)